In [1]:
from surgeon_recording.reader import Reader
from glob import glob
from os.path import join
import os
import numpy as np
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from copy import deepcopy
import pandas as pd

import cv2
import csv
from natsort import natsorted
from surgeon_recording.synchronization import Synchro

from moviepy.editor import *

from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import moviepy.video.io.ImageSequenceClip

In [2]:
# code to export from absolute time, to be integrated afterwards with the microscope frames giving the absolute time
synchro = Synchro()

recording_session = 'test_19_04_22'
subject = 'ANAELLE_all4'


#path_to_data_folder = '/home/anaelle/git/surgeon_recording/data'
path_to_data_folder = '/Users/anael/Documents/GitHub/surgeon_recordings/data'

data_folder = join('..', 'data', recording_session , subject)

In [3]:
sensors = ['optitrack', 'emg', 'tps']

In [17]:
# get sensor data from CSV

index_sensor = {}
relative_time_sensor = {}
absolute_time_sensor = {}
headers = {}
directory_sensor = {}


#file = [x[2] for x in os.walk(join(data_folder, 'SENSORS'))]
#print(file[0])  

for sensor in sensors:
    file = sensor + '.csv'
    print(file)
    directory_sensor[sensor] = join(data_folder, 'SENSORS', file)

    synchro.play(directory_sensor[sensor], sensor)
    index_sensor[sensor] = synchro.data[sensor]['index']
    relative_time_sensor[sensor] = synchro.data[sensor]['relative_time']
    absolute_time_sensor[sensor] = synchro.data[sensor]['absolute_time']
    
    f = open(directory_sensor[sensor])
    d_reader = csv.DictReader(f)
    #get fieldnames from DictReader object and store in list
    headers[sensor] = d_reader.fieldnames
    print(headers[sensor])


optitrack.csv
['index', 'absolute_time', 'relative_time', 'tweezers_x', 'tweezers_y', 'tweezers_z', 'tweezers_qx', 'tweezers_qy', 'tweezers_qz', 'tweezers_qw', 'scissors_x', 'scissors_y', 'scissors_z', 'scissors_qx', 'scissors_qy', 'scissors_qz', 'scissors_qw']
emg.csv
['index', 'absolute_time', 'relative_time', 'emg0', 'emg1', 'emg2', 'emg3', 'emg4', 'emg5', 'emg6', 'emg7']
tps.csv
['index', 'absolute_time', 'relative_time', 'right_thumb_calibrated', 'right_thumb_raw', 'right_index_calibrated', 'right_index_raw', 'right_middle_calibrated', 'right_middle_raw']


In [78]:
camera_ref = "MICROSCOPE"

#get ref frame of microscope directly from csv file
file = [x[2] for x in os.walk(join(data_folder, 'SEGMENTATION_CSV'))]
print(file[0][0])  
    
directory = join(data_folder ,'SEGMENTATION_CSV', file[0][0])
print(directory)
DATA = pd.read_csv(directory)
start_ref_frame_vector = DATA['Start_frame']
stop_ref_frame_vector = DATA['Stop_frame']
index_segment_vector = DATA['nb']
#print(start_ref_frame)
    
for o in range(len(start_ref_frame_vector)):                                     #pour chaque segment
    start_ref_frame = start_ref_frame_vector[o]
    stop_ref_frame = stop_ref_frame_vector[o]

    #camera = camera_ref
    #start_ref_abs_time = absolute_time[camera_ref][start_ref_frame]
    #stop_ref_abs_time = absolute_time[camera_ref][stop_ref_frame]
    start_ref_abs_time = 1650380304.000000
    stop_ref_abs_time = 1650380305.200000
    
    pos_start_sensor = {}
    pos_stop_sensor = {}
    #rel_time_start = {}
    #rel_time_stop = {}
    abs_time_start_sensor = {}
    abs_time_stop_sensor = {}

    print("start reference abs time:  ", start_ref_abs_time)  
    print("stop reference abs time:  ", stop_ref_abs_time) 
    print("----------------------")
    for sensor in sensors:
        pos_start_sensor[sensor] = []
        pos_stop_sensor[sensor] = []
        abs_time_start_sensor[sensor] = {}
        abs_time_stop_sensor[sensor] = {}
        
        [pos_start_sensor[sensor], abs_time_start_sensor[sensor]]= synchro.take_closest(absolute_time_sensor[sensor], start_ref_abs_time, 0)
        print(sensor, "start index:       ", pos_start_sensor[sensor], "     abs time:   ", abs_time_start_sensor[sensor])
        [pos_stop_sensor[sensor], abs_time_stop_sensor[sensor]] = synchro.take_closest(absolute_time_sensor[sensor], stop_ref_abs_time, 0)
        print(sensor, "stop index:        ", pos_stop_sensor[sensor], "      abs time:   ", abs_time_stop_sensor[sensor])
        
        r1 = index_segment_vector[o]
        ref = "segment_nb_" + str(r1) + "_" + sensor + ".csv"
        csv_name = join(data_folder,'SENSORS', 'SEGMENTATION', ref)
          
        f = open(csv_name, 'w', newline='') # open the file in the write mode                        #open last one
    
        # create the csv writer
        writer = csv.writer(f)
       
        # write a row to the csv file
        writer.writerow(headers[sensor])
        table = pd.read_csv(directory_sensor[sensor],skiprows=pos_start_sensor[sensor]-1,nrows=(pos_stop_sensor[sensor] - pos_start_sensor[sensor]+1))
        table2 = table.to_numpy()
    
        for m in range(pos_stop_sensor[sensor] - pos_start_sensor[sensor]+1):
            row = table2[m]
            writer.writerow(row)
            # print(row)
    
        f.close()        


segmentation_micro_frames.csv
..\data\test_19_04_22\ANAELLE_all4\SEGMENTATION_CSV\segmentation_micro_frames.csv
start reference abs time:   1650380304.0
stop reference abs time:   1650380305.2
----------------------
optitrack start index:        59      abs time:    1650380303.997059
optitrack stop index:         148       abs time:    1650380305.206076
emg start index:        714      abs time:    1650380304.000162
emg stop index:         1811       abs time:    1650380305.2000115
tps start index:        16      abs time:    1650380303.9820645
tps stop index:         51       abs time:    1650380305.1265695


In [ ]:
table2



,nb;Start_frame;Stop_frame
0,1;400;500


In [ ]:
 DATA['nb']